In [1]:
from IPython.display import clear_output
import random

def deck_setup():                          ##Procedure to initialise the deck
    global deck
    deck = []
    
    global colours
    colours = ["Red", "Blue", "Green", "Yellow"]
    
    for i in colours:
        for u in range(2):
            deck.append((i, "Draw 2"))     ##Adds two +2 cards to the deck in each colour
            deck.append((i, "Reverse"))    ##Adds two reverse cards to the deck in each colour
            deck.append((i, "Skip"))       ##Adds two skip cards to the deck in each colour

        for j in range(10):
            deck.append((i, str(j)))       ##Adds numbered cards from 0 to 9 to the deck in each colour

    deck.append(("Wild", "") * 4)          ##Adds four wild cards to the deck
    deck.append(("Wild", "Draw 4") * 4)    ##Adds four wild +4 cards to the deck
    
    random.shuffle(deck)                   ##Shuffles deck

    
    
    
    
class Player:
    def __init__(self, number, hand):
        self.number = number
        self.hand = hand
        
##Initialises players
Player1 = Player(1, [])
Player2 = Player(2, [])
Player3 = Player(3, [])
Player4 = Player(4, [])

##Initialises player list
player_list = [Player1, Player2, Player3, Player4]

##data validation for number of players
while True:
    number_of_players = input("How many players? (2 - 4): ")     ##takes in input of number of players as a string
    if number_of_players.isdigit() == True and int(number_of_players) in range(2, 5):
        player_list = player_list[:int(number_of_players)]       ##slices player_list to fit the total number of players   
        break
    else:
        print("Error, invalid input.")

clear_output()
        
deck_setup()       

for i in range(7):
    for j in player_list:
        j.hand.append(deck.pop(0))     ##Creates hand of each player
        
discard_pile = []
for i in range(len(deck)):
    if deck[i][0] in colours and deck[i][1].isdigit() == True:
        discard_pile.append(deck.pop(i))                         ## initialises the first card on the discard pile and makes sure it is not an action/wild card
        break

##Reverses the player order pivotig around the player that played the reverse card
def reverse(player):
    global player_list
    
    if len(player_list) == 2:
        global skip
        skip = True
    
    player_index = player_list.index(player)
    player_list = player_list[::-1]
    
    while player_list.index(player) != player_index:
        temp_player = player_list.pop()
        player_list.insert(0, temp_player)
        
    print("\nThe turn order has been reversed!")




##Adds a number of cards to the next player's hand    
def add(player, card):
    global player_list
    
    player_index = player_list.index(player)
    if player_index == len(player_list) - 1:
        player_index = 0
    else:
        player_index += 1

    draw_player = player_list[player_index]
    
    for i in range(int(card[1][-1])):
        draw_player.hand.append(deck.pop(0))
        
    print(f"\nPlayer {draw_player.number} drew {card[1][-1]} cards!")
    
    global skip
    skip = True
    print(f"\nPlayer {draw_player.number}'s turn has been skipped!")
    


    

##Changes the colour of the wild card, if a Wild +4 Card is played it will also add 4 cards to the next player's hand
def wild(player, card):
    print("")
    print("1. Red\n2. Blue\n3. Green\n4. Yellow")
    
    while True:
        colour_input = input("Pick a colour (1 - 4): ")
            
        if colour_input in ["1", "2", "3", "4"]:
            break
        else:
            print("Error, invalid input.")
    
    clear_output()
    
    print(f"\nThe current colour is now {colours[int(colour_input) - 1]}!")
    temp_card = list(card)
    temp_card[0] = colours[int(colour_input) - 1]
    card = tuple(temp_card)
    
    
    if card[1] == "Draw 4":
        add(player, card)
    
    return card


    
    
    
def play_card(player, index, card):
    global player_list
    
    if card[0] == "Wild":                                                  ##checks for if the card is a wild card
        card_coloured = wild(player, card)
        discard_pile.insert(0, card_coloured)                              ##Removes card from player's hand and adds it to the discard pile    
        return True
    elif card[0] == discard_pile[0][0] or card[1] == discard_pile[0][1]:   ##checks for if the colour/number/action of the card matches
        clear_output()
        
        if len(player.hand) == 1:
            print("Uno!")

        if card[1] == "Reverse":                                           ##Reverses the player order
            reverse(player)  
        elif card[1] == "Skip":                                            ##Skips the next player's turn (work in progress)
            global skip
            skip = True
            
            player_index = player_list.index(player)
            if player_index == len(player_list) - 1:
                player_index = 0
            else:
                player_index += 1

            skipped_player = player_list[player_index]
                
            print(f"\nPlayer {skipped_player.number}'s turn has been skipped!")
        elif card[1] == "Draw 2":                                          ##Adds two cards to the next player's hand and skips their turn
            add(player, card)

        discard_pile.insert(0, card)                                       ##Removes card from player's hand and adds it to the discard pile
        return True
    else:
        print("Sorry but you need to match either by the colour, number or action.")
        return False
    
    
    
    
    
def play_hand(player):
    while True:
        ##prints all of the playable cards in the player's hand
        for i in range(len(player.hand)):
            card = player.hand[i]
            print(f"{i + 1}. {card[0]} {card[1]}")

        ##data validation for card input
        while True:
            card_input = input(f"Select an Card (1 - {len(player.hand)}, 0 to return to main menu): ")    ##takes in card input as a string 
            
            if card_input == "0":                                                                         ##Input 0 to quit and return to main interface
                return False

            if card_input.isdigit() == True and int(card_input) in range(1, len(player.hand) + 1):
                chosen_card = player.hand.pop(int(card_input) - 1)
                valid_card = play_card(player, int(card_input), chosen_card)
                
                if valid_card == True:
                    return True
                else:
                    player.hand.insert(int(card_input) - 1, chosen_card)
            else:
                print("Error, invalid input.")

                                    
                                    
                                    
                                    
def main_interface(player):
    menu = True
    while menu == True:
        print("\nWhat would you like to do? \n1. Look at your Hand \n2. Draw a card")     ##user interface for start of player turn

        ##data validation for player input
        while True:
            player_input = input("Select an Option (1 or 2): ")                         ##takes in player input as a string
            print("")
            if player_input == "1" or player_input == "2":
                break
            else:
                print("Error, invalid input.")

        ##main decision making process (1 shows the hand of the player, 2 draws a card from the deck and adds it to the player's hand)
        if player_input == "1":
            play = play_hand(player)
            if play == True:
                break

        elif player_input == "2":
            player.hand.append(deck.pop(0))                                             ##adds a card from the deck to the player's hand
            new_card = player.hand[-1]
            input(f"You drew a {new_card[0]} {new_card[1]}.")

            if new_card[0] == "Wild" or new_card[0] == discard_pile[0][0] or  new_card[1] == discard_pile[0][1]:
                print("Do you want to play this card?")

                while True:
                    play_drawcard_input = input("Enter Y or N: ").upper()

                    if play_drawcard_input == "Y":
                        new_card = player.hand.pop(-1)
                        play_card(player, len(player.hand), new_card)
                        clear_output()
                        break
                    elif play_drawcard_input == "N":
                        clear_output()
                        break
                    else:
                        print("Error, invalid input.")
                
                if len(player.hand) == 1:
                    print("Uno!")
            else:
                clear_output()
    
            break

    

    
    
skip = False
win_cond = False

while win_cond == False:
    
    ##Skipped player check
    for i in range(len(player_list)):
        if skip == True:
            skip = False
            continue
    
        current_player = player_list[i]
        
        ##Formatting of a recently colour changed Wild card to make it more readable to the users
        if discard_pile[0][1] == "":
            temp_card = list(discard_pile[0])
            temp_card[1] = "Wild"
            discard_pile[0] = tuple(temp_card)

        print(f"\nThe current card is a {discard_pile[0][0]} {discard_pile[0][1]}.")     ##Displays message of the current card
        print(f"It's Player {current_player.number}'s turn!")                            ##Displays the player turn
        main_interface(current_player)
        
        ##Checks for if the player has anymore cards (0 cards means the player wins)
        if len(current_player.hand) == 0:
            clear_screen()
            
            win_cond = True
            
            print("")
            print(f"Player {current_player.number} wins the game!\nThanks for playing!")     ##Displays message of the winning player and ends the game
                
            break
        
        ##Increases the number of cards in the deck using cards from the discard pile when len(deck) falls below 6
        if len(deck) <= 5:
            temp_list = discard_pile[1:]
            random.shuffle(temp_list)
            
            deck.extend(temp_list)
            del discard_pile[1:]

IndentationError: unexpected indent (183766255.py, line 166)